In [ ]:
%load_ext lab_black

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
from pmdarima.model_selection import train_test_split
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import joblib

path_data = here("./data")
os.chdir(path_data)
data_iu_analysis = pd.read_parquet("data_iu_analysis.parquet")

In [ ]:
test_size = round(data_iu_analysis.shape[0] * 0.2)

In [ ]:
train, test = train_test_split(data_iu_analysis, test_size=test_size)

In [ ]:
x_train = train.drop(columns="tournament_make")
x_test = test.drop(columns="tournament_make")
y_train = train["tournament_make"]
y_test = test["tournament_make"]

In [ ]:
%pip install xgboost==1.7.3

In [ ]:
print(xgb.__version__)

In [ ]:
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


def objective(trial):
    # Define the hyperparameters to search over
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_discrete_uniform("subsample", 0.5, 0.9, 0.1),
        "colsample_bytree": trial.suggest_discrete_uniform(
            "colsample_bytree", 0.5, 0.9, 0.1
        ),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "objective": "binary:logistic",
        "random_state": 42,
    }

    # Train the XGBoost model with the given hyperparameters
    model = xgb.XGBClassifier(**params)
    model.fit(x_train, y_train)

    # Evaluate the model on the test set and return the accuracy as the objective value
    preds = model.predict(x_train)
    accuracy = accuracy_score(y_train, preds)
    return accuracy


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found by Optuna
print("Best hyperparameters:", study.best_params)

# Train the final model with the best hyperparameters on the full dataset
best_model = xgb.XGBClassifier(**study.best_params)
best_model.fit(x_train, y_train)

In [ ]:
path_outputs = here("./outputs")
os.chdir(path_outputs)
joblib.dump(best_model, "model_iu_bball.jlib")

In [ ]:
prob = best_model.predict_proba(x_test)[:, 1]

In [ ]:
prob

In [ ]:
from sklearn.metrics import f1_score


def find_best_threshold(y_true, y_pred_prob):
    """
    Find the best threshold to use for converting predicted probabilities to binary predictions,
    using the F1 score as the metric to optimize for.

    :param y_true: true labels, array-like of shape (n_samples,)
    :param y_pred_prob: predicted probabilities, array-like of shape (n_samples,)
    :return: best threshold value
    """
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    best_f1 = 0
    best_threshold = 0
    for threshold in thresholds:
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

In [ ]:
threshold = find_best_threshold(y_true=y_test, y_pred_prob=prob)
threshold

In [ ]:
prob_threshold = np.where(prob > threshold, 1, 0)

In [ ]:
from sklearn.metrics import balanced_accuracy_score, f1_score

In [ ]:
balanced_accuracy = balanced_accuracy_score(y_true=y_test, y_pred=prob_threshold)
balanced_accuracy

In [ ]:
f1_accuracy = f1_score(y_true=y_test, y_pred=prob_threshold)
f1_accuracy